In [ ]:
'Baby presence detection with obstructions like Sunscreen and Blanket'

In [ ]:
import gc
gc.collect()

In [ ]:
%reset -f

In [ ]:
# Import every packages for EDA

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Data Preprocessing

# Add measurements for withoutbaby

In [ ]:
import numpy as np
import pandas as pd
import os 
# Replace 'your_array_file.npy' with the actual file path of your saved NumPy array
#file_path = 'Carrierseat_withoutbaby_npy_array_Lowpassfiltered_label_0.npy'

# Path to the file (same as used when saving)
file_path = os.path.join("..", "..", "Data", "Processed",
    "CarrierSeat_withoutBaby_Lowpassfilered_Label_0.npy")


# Load the NumPy array from the file
loaded_array = np.load(file_path, mmap_mode='r')

# Now 'loaded_array' contains the NumPy array data that was saved in the file
dataframe_withoutbaby = pd.DataFrame(loaded_array,columns=['Frequency','FFT Magnitude','Phase','Infant_Presence'])
dataframe_withoutbaby

In [ ]:
num_rows_withoutbaby = len(dataframe_withoutbaby)
num_rows_withoutbaby

In [ ]:
fft_magnitudes_withoutbaby = dataframe_withoutbaby["FFT Magnitude"].values  # Convert to NumPy array
fft_magnitudes_withoutbaby

In [ ]:
fft_frequencies_withoutbaby = dataframe_withoutbaby["Frequency"].values  # Convert to NumPy array
fft_frequencies_withoutbaby

In [ ]:
fft_phase_withoutbaby = dataframe_withoutbaby["Phase"].values  # Convert to NumPy array
fft_phase_withoutbaby

In [ ]:
import numpy as np
from scipy.stats import entropy
from scipy.signal import find_peaks

def extract_features(fft_freqs, fft_mags, fft_phase):
    # Normalize magnitudes to prevent division errors
    norm_mags = fft_mags / np.sum(fft_mags) if np.sum(fft_mags) > 0 else fft_mags
    
    # Spectral Centroid (Weighted Mean of Frequencies)
    spectral_centroid = np.sum(fft_freqs * norm_mags) / np.sum(norm_mags)

    # Spectral Bandwidth (Spread around centroid)
    spectral_bandwidth = np.sqrt(np.sum(norm_mags * (fft_freqs - spectral_centroid) ** 2))

    # Spectral Flatness (Geometric Mean / Arithmetic Mean)
    spectral_flatness = np.exp(np.mean(np.log(fft_mags + 1e-10))) / np.mean(fft_mags + 1e-10)

    # Find peaks in the magnitude spectrum
    peaks, _ = find_peaks(fft_mags, height=0.1 * np.max(fft_mags))  # Adaptive threshold

    # Harmonic Ratio (Ratio of 2nd peak to 1st peak)
    if len(peaks) >= 2:
        harmonic_ratio = fft_mags[peaks[1]] / fft_mags[peaks[0]]
    else:
        harmonic_ratio = 0  # No second peak detected

    # Phase Features
    phase_variance = np.var(fft_phase)  # Variance of phase angles
    phase_mean = np.mean(fft_phase)  # Mean of phase angles
    phase_diff = np.mean(np.diff(fft_phase))  # Average phase difference between frequencies

    return {
        "mean_fft": np.mean(fft_mags),
        "std_fft": np.std(fft_mags),
        "max_fft": np.max(fft_mags),
        "min_fft": np.min(fft_mags),
        "median_fft": np.median(fft_mags),
        "sum_fft": np.sum(fft_mags),
        "spectral_entropy": entropy(norm_mags),  # Energy spread
        "spectral_centroid": spectral_centroid,
        "spectral_bandwidth": spectral_bandwidth,
        "spectral_flatness": spectral_flatness,
        "phase_variance": phase_variance,
        "phase_mean": phase_mean,
        "phase_diff": phase_diff,
    }


In [ ]:
# Function to apply perturbation based on reference values
def add_perturbation(reference_features, num_rows, perturb_range=0.03):
    perturbed_data = []
    
    for _ in range(num_rows):
        perturbed_features = {
            key: value * (1 + np.random.uniform(-perturb_range, perturb_range)) 
            for key, value in reference_features.items()
        }
        perturbed_data.append(perturbed_features)
    
    return pd.DataFrame(perturbed_data)

In [ ]:

# Extract features for fft_withoutbaby
reference_features_withoutbaby = extract_features(fft_frequencies_withoutbaby, fft_magnitudes_withoutbaby, fft_phase_withoutbaby)
reference_features_withoutbaby



In [ ]:
# Generate a DataFrame with perturbed values for all rows in fft_withoutbaby
num_rows_withoutbaby = len(dataframe_withoutbaby)
perturbed_withoutbaby_df = add_perturbation(reference_features_withoutbaby, num_rows_withoutbaby, perturb_range=0.03)
perturbed_withoutbaby_df

In [ ]:
dataframe_withoutbaby

In [ ]:
len(perturbed_withoutbaby_df),len(dataframe_withoutbaby)

In [ ]:

# Ensure both DataFrames have the same number of rows
if len(dataframe_withoutbaby) != len(perturbed_withoutbaby_df):
    print("The DataFrames have different lengths. Adjust them to have the same number of rows before concatenating.")
else:
    # Concatenate the DataFrames side by side
    dataframe_withoutbaby_withfeatures = pd.concat([dataframe_withoutbaby, perturbed_withoutbaby_df], axis=1)

# Display the first few rows of the updated dataframe_withoutbaby to verify
dataframe_withoutbaby_withfeatures

# Withbaby covered with blanket and sunscreen measurements 

In [ ]:
import numpy as np
import pandas as pd
import os

# Replace 'your_array_file.npy' with the actual file path of your saved NumPy array
file_path = os.path.join("..", "..", "Data", "Processed",
    "BabyCovered_with_Blanket_or_Sunscreen_npy_array_Lowpassfiltered_label.npy")

# Load the NumPy array from the file
loaded_array = np.load(file_path, mmap_mode='r')

# Now 'loaded_array' contains the NumPy array data that was saved in the file
dataframe_withbaby = pd.DataFrame(loaded_array,columns=['Frequency','FFT Magnitude','Phase','Infant_Presence'])
dataframe_withbaby

In [ ]:
num_rows_withbaby = len(dataframe_withbaby)
num_rows_withbaby

In [ ]:
fft_magnitudes_withbaby = dataframe_withbaby["FFT Magnitude"].values  # Convert to NumPy array
fft_magnitudes_withbaby

In [ ]:
fft_frequencies_withbaby = dataframe_withbaby["Frequency"].values  # Convert to NumPy array
fft_frequencies_withbaby

In [ ]:
fft_phase_withbaby = dataframe_withbaby["Phase"].values  # Convert to NumPy array
fft_phase_withbaby

In [ ]:
# Extract features for fft_withbaby
reference_features_withbaby = extract_features(fft_frequencies_withbaby, fft_magnitudes_withbaby, fft_phase_withbaby)
reference_features_withbaby



In [ ]:
# Generate a DataFrame with perturbed values for all rows in fft_withoutbaby
num_rows_withbaby = len(dataframe_withbaby)
perturbed_withbaby_df = add_perturbation(reference_features_withbaby, num_rows_withbaby, perturb_range=0.03)
perturbed_withbaby_df

In [ ]:
dataframe_withbaby

In [ ]:
len(perturbed_withbaby_df),len(dataframe_withbaby)

In [ ]:

# Ensure both DataFrames have the same number of rows
if len(dataframe_withbaby) != len(perturbed_withbaby_df):
    print("The DataFrames have different lengths. Adjust them to have the same number of rows before concatenating.")
else:
    # Concatenate the DataFrames side by side
    dataframe_withbaby_withfeatures = pd.concat([dataframe_withbaby, perturbed_withbaby_df], axis=1)

# Display the first few rows of the updated dataframe_withoutbaby to verify
dataframe_withbaby_withfeatures

In [ ]:
dataframe_withoutbaby

In [ ]:
dataframe_withbaby

In [ ]:
# Vertical concatenation
dataset_2_rawdata = pd.concat([dataframe_withoutbaby,dataframe_withbaby])
dataset_2_rawdata

In [ ]:
# Resetting the index if unique indices are needed
dataset_2_rawdata = dataset_2_rawdata.reset_index(drop=True)
dataset_2_rawdata

In [ ]:
dataframe_withbaby_withfeatures

In [ ]:
dataframe_withoutbaby_withfeatures

In [ ]:
# Vertical concatenation
dataset_2_withfeatures = pd.concat([dataframe_withoutbaby_withfeatures,dataframe_withbaby_withfeatures])
dataset_2_withfeatures

In [ ]:
# Resetting the index if unique indices are needed
dataset_2_withfeatures = dataset_2_withfeatures.reset_index(drop=True)
dataset_2_withfeatures

# Training with rawdata

In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# Load your dataset (df_combined)
X = dataset_2_rawdata.drop(columns=['Infant_Presence'])  # Features
y = dataset_2_rawdata['Infant_Presence']  # Labels

# Split into training & testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the features (XGBoost handles unscaled data well, but scaling can help)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Convert data into DMatrix format (optimized for XGBoost)
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

# Set up XGBoost parameters
params = {
    'objective': 'binary:logistic',  # For binary classification
    'eval_metric': 'logloss',        # Loss function
    'max_depth': 6,                   # Depth of trees
    'learning_rate': 0.1,              # Step size shrinkage
    'n_estimators': 1000,               # Number of trees
    'random_state': 42
}

# Train XGBoost model
clf = xgb.XGBClassifier(**params)
clf.fit(X_train, y_train)

# Make predictions
y_pred = clf.predict(X_test)

# Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report for rawdata:\n", classification_report(y_test, y_pred))

In [ ]:
# Generate confusion matrix
cm = confusion_matrix(y_test, y_pred)
cm

fig, ax = plt.subplots(figsize=(6, 6))
sns.heatmap(cm, annot=True, fmt='d', ax=ax)  # Use fmt='d' for integer annotation
plt.xlabel("Predicted label")
plt.ylabel("True label")
plt.title('Confusion Matrix Baby presence detection when covered in Blanket or sunscreen- XG Boost Model with rawdata')
plt.show()

# Training with extracted features

In [ ]:
# XG Boost Model with extracted features data

In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# Load your dataset (df_combined)
X = dataset_2_withfeatures.drop(columns=['Infant_Presence'])  # Features
y = dataset_2_withfeatures['Infant_Presence']  # Labels

# Split into training & testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the features (XGBoost handles unscaled data well, but scaling can help)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Convert data into DMatrix format (optimized for XGBoost)
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

# Set up XGBoost parameters
params = {
    'objective': 'binary:logistic',  # For binary classification
    'eval_metric': 'logloss',        # Loss function
    'max_depth': 6,                   # Depth of trees
    'learning_rate': 0.1,              # Step size shrinkage
    'n_estimators': 1000,               # Number of trees
    'random_state': 42
}

# Train XGBoost model
clf = xgb.XGBClassifier(**params)
clf.fit(X_train, y_train)

# Make predictions
y_pred = clf.predict(X_test)

# Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report for extracted features data:\n", classification_report(y_test, y_pred))

In [ ]:
# Generate confusion matrix
cm = confusion_matrix(y_test, y_pred)
cm

fig, ax = plt.subplots(figsize=(6, 6))
sns.heatmap(cm, annot=True, fmt='d', ax=ax)  # Use fmt='d' for integer annotation
plt.xlabel("Predicted label")
plt.ylabel("True label")
plt.title('Confusion Matrix Baby presence detection when covered in Blanket or sunscreen - XG Boost Model with extracted features')
plt.show()

# Feature selection for overcoming OVERFITTING

In [ ]:
# Plotting correlation matrix for extracted features
corr_matrix = dataset_2_withfeatures.corr()
fig= plt.subplots(figsize=(15, 10))
ax= sns.heatmap(corr_matrix,
               annot=True,
               linewidths=0.5,
               fmt='.2f',
               cmap='YlGnBu')



In [ ]:
# Set a threshold for correlation (e.g., 0.8)
threshold = 0.8

# Find pairs of highly correlated features
highly_correlated = np.where(np.abs(corr_matrix) > threshold)

# Create a set to store the columns to drop
to_drop = set()

# Loop through the indices of the highly correlated pairs
for i, j in zip(*highly_correlated):
    if i != j:  # Avoid diagonal (self-correlation)
        feature_i = corr_matrix.columns[i]
        feature_j = corr_matrix.columns[j]
        # Ensure we do NOT drop 'Object_Presence'
        if feature_j != "Infant_Presence":
            to_drop.add(feature_j)  # Drop one of the correlated features

# Drop the highly correlated features from the dataframe
dataset_2_reduced = dataset_2_withfeatures.drop(columns=to_drop)

# Display the dropped features and new dataframe shape
print("Dropped features: ", to_drop)
print("New dataframe shape: ", dataset_2_reduced.shape)


In [ ]:
dataset_2_reduced

In [ ]:
corr_matrix = dataset_2_reduced.corr()
fig= plt.subplots(figsize=(15, 10))
ax= sns.heatmap(corr_matrix,
               annot=True,
               linewidths=0.5,
               fmt='.2f',
               cmap='YlGnBu')

# XG BOOST with selected features using Correlation matrix

In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Load your dataset (df_combined)
X = dataset_2_reduced.drop(columns=['Infant_Presence'])  # Features
y = dataset_2_reduced['Infant_Presence']  # Labels

# Split into training & testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the features (XGBoost handles unscaled data well, but scaling can help)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Convert data into DMatrix format (optimized for XGBoost)
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

# Set up XGBoost parameters
params = {
    'objective': 'binary:logistic',  # For binary classification
    'eval_metric': 'logloss',        # Loss function
    'max_depth': 6,                   # Depth of trees
    'learning_rate': 0.1,              # Step size shrinkage
    'n_estimators': 1000,               # Number of trees
    'random_state': 42
}

# Train XGBoost model
clf = xgb.XGBClassifier(**params)
clf.fit(X_train, y_train)

# Make predictions
y_pred = clf.predict(X_test)

# Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report for selected features data:\n", classification_report(y_test, y_pred))



In [ ]:
# Generate confusion matrix
cm = confusion_matrix(y_test, y_pred)
cm
fig, ax = plt.subplots(figsize=(6, 6))
sns.heatmap(cm, annot=True, fmt='d', ax=ax)  # Use fmt='d' for integer annotation
plt.xlabel("Predicted label")
plt.ylabel("True label")
plt.title('Confusion Matrix Baby presence detection when covered in Blanket or sunscreen -  XG Boost Model - selected features')
plt.show()

In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score
import matplotlib.pyplot as plt

# Get predicted probabilities
y_prob = clf.predict_proba(X_test)[:, 1]  # Probability of positive class

# Compute ROC curve values
fpr, tpr, thresholds = roc_curve(y_test, y_prob)

# Compute AUC score
auc_score = roc_auc_score(y_test, y_prob)
auc_score
# Plot ROC Curve
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, label=f"ROC Curve (AUC = {auc_score:.2f})", color='blue')
plt.plot([0, 1], [0, 1], 'k--', label="Random Guessing (AUC = 0.5)")
plt.xlabel("False Positive Rate (FPR)")
plt.ylabel("True Positive Rate (TPR)")
plt.title("ROC Curve")
plt.legend()
plt.show()


In [ ]:
# Feature Importance Plot
importances = clf.feature_importances_
feature_names = X.columns
importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': importances})
importance_df = importance_df.sort_values(by="Importance", ascending=False)

plt.figure(figsize=(8, 6))
sns.barplot(x=importance_df["Importance"], y=importance_df["Feature"], palette="viridis")
plt.xlabel("Feature Importance")
plt.ylabel("Feature")
plt.title("Feature Importance Chart - Random Forest")
plt.show()


In [ ]:
import os
import joblib

# Save the trained XGBoost model
# clf.save_model('xgboost_model_Baby_Covered_in_Blanket_Sunscreen.json')

# Save the scaler
# joblib.dump(scaler, 'xgboost_scaler_Baby_Covered_in_Blanket_Sunscreen.pkl')

# Define save directories (2 levels up)
misc_dir = os.path.join("..", "..", "Misc")
model_dir = os.path.join("..", "..", "Models", "Baby Detection and Seat Classification XGBoost")


# Define file paths
model_path = os.path.join(misc_dir, "xgboost_model_Baby_Covered_in_Blanket_Sunscreen.json")
scaler_path = os.path.join(model_dir, "xgboost_scaler_Baby_Covered_in_Blanket_Sunscreen.pkl")

# --- Check and replace model ---
if os.path.exists(model_path):
    os.remove(model_path)
    print(f"Old model removed: {model_path}")

clf.save_model(model_path)
print(f"New model saved at: {os.path.abspath(model_path)}")

# --- Check and replace scaler ---
if os.path.exists(scaler_path):
    os.remove(scaler_path)
    print(f"Old scaler removed: {scaler_path}")

joblib.dump(scaler, scaler_path)
print(f"New scaler saved at: {os.path.abspath(scaler_path)}")


In [ ]:

import xgboost as xgb

# Load the trained XGBoost model
#clf = xgb.XGBClassifier()
#clf.load_model('xgboost_model_Baby_Covered_in_Blanket_Sunscreen.json')

# Load the scaler
#scaler = joblib.load('xgboost_scaler_Baby_Covered_in_Blanket_Sunscreen.pkl')

# Example: Transform new data and make predictions
#X_new_scaled = scaler.transform(X_new)
#y_pred = clf.predict(X_new_scaled)


# Random Forest Model

In [ ]:
# Random Forest Model with raw data

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA  # Import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

X = dataset_2_rawdata.drop('Infant_Presence',axis=1)
Y = dataset_2_rawdata['Infant_Presence']

# Split into training & testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42, shuffle=True)

# Train a Random Forest Classifier
clf = RandomForestClassifier(n_estimators=1000, max_depth=10, min_samples_split = 2, min_samples_leaf = 2, max_features = "sqrt", random_state=42)
clf.fit(X_train, y_train)

# Make predictions
y_pred = clf.predict(X_test)

# Evaluate the model
print("RanFor Accuracy for rawdata:", accuracy_score(y_test, y_pred))
print("Classification Report for rawdata:\n", classification_report(y_test, y_pred))

In [ ]:
cm =confusion_matrix(y_test,y_pred)
sns.set(font_scale=1)
fig, ax = plt.subplots(figsize=(6, 6))
sns.heatmap(cm, annot=True, fmt='d', ax=ax)  # Use fmt='d' for integer annotation
plt.xlabel("Predicted label")
plt.ylabel("True label")
plt.title('Confusion Matrix Baby presence detection when covered in Blanket or sunscreen - RanFor - rawdata ')
plt.show()



In [ ]:
# Random forest Model with extracted features data

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA  # Import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

X = dataset_2_withfeatures.drop('Infant_Presence',axis=1)
Y = dataset_2_withfeatures['Infant_Presence']

# Split into training & testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42, shuffle=True)

# Train a Random Forest Classifier
clf = RandomForestClassifier(n_estimators=1000, max_depth=10, min_samples_split = 2, min_samples_leaf = 2, max_features = "sqrt", random_state=42)
clf.fit(X_train, y_train)

# Make predictions
y_pred = clf.predict(X_test)

# Evaluate the model
print("RanFor Accuracy for extracted features data:", accuracy_score(y_test, y_pred))
print("Classification Report for extracted features data:\n", classification_report(y_test, y_pred))

In [ ]:
cm =confusion_matrix(y_test,y_pred)
sns.set(font_scale=1)
fig, ax = plt.subplots(figsize=(6, 6))
sns.heatmap(cm, annot=True, fmt='d', ax=ax)  # Use fmt='d' for integer annotation
plt.xlabel("Predicted label")
plt.ylabel("True label")
plt.title('Confusion Matrix Baby presence detection when covered in Blanket or sunscreen - RanFor - extracted features data ')
plt.show()

In [ ]:
# Random forest Model with selected features data

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA  # Import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

X = dataset_2_reduced.drop('Infant_Presence',axis=1)
Y = dataset_2_reduced['Infant_Presence']

# Split into training & testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42, shuffle=True)

# Train a Random Forest Classifier
clf = RandomForestClassifier(n_estimators=1000, max_depth=10, min_samples_split = 2, min_samples_leaf = 2, max_features = "sqrt", random_state=42)
clf.fit(X_train, y_train)

# Make predictions
y_pred = clf.predict(X_test)

# Evaluate the model
print("RanFor Accuracy for selected features data:", accuracy_score(y_test, y_pred))
print("Classification Report for selected features data:\n", classification_report(y_test, y_pred))

In [ ]:
cm =confusion_matrix(y_test,y_pred)
sns.set(font_scale=1)
fig, ax = plt.subplots(figsize=(6, 6))
sns.heatmap(cm, annot=True, fmt='d', ax=ax)  # Use fmt='d' for integer annotation
plt.xlabel("Predicted label")
plt.ylabel("True label")
plt.title('Confusion Matrix Baby presence detection when covered in Blanket or sunscreen - RanFor - selected features ')
plt.show()


In [ ]:
# Feature Importance Plot
importances = clf.feature_importances_
feature_names = X.columns
importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': importances})
importance_df = importance_df.sort_values(by="Importance", ascending=False)

plt.figure(figsize=(8, 6))
sns.barplot(x=importance_df["Importance"], y=importance_df["Feature"], palette="viridis")
plt.xlabel("Feature Importance")
plt.ylabel("Feature")
plt.title("Feature Importance Chart - Random Forest")
plt.show()


In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score
import matplotlib.pyplot as plt

# Get predicted probabilities
y_prob = clf.predict_proba(X_test)[:, 1]  # Probability of positive class

# Compute ROC curve values
fpr, tpr, thresholds = roc_curve(y_test, y_prob)

# Compute AUC score
auc_score = roc_auc_score(y_test, y_prob)
auc_score
# Plot ROC Curve
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, label=f"ROC Curve (AUC = {auc_score:.2f})", color='blue')
plt.plot([0, 1], [0, 1], 'k--', label="Random Guessing (AUC = 0.5)")
plt.xlabel("False Positive Rate (FPR)")
plt.ylabel("True Positive Rate (TPR)")
plt.title("ROC Curve")
plt.legend()
plt.show()


In [ ]:

import os
import joblib

# Save the model
# joblib.dump(clf, 'random_forest_model_Baby_Covered_in_Blanket_Sunscreen.pkl')

# Define save directory (2 levels up -> Model/Baby Covered in Blanket Sunscreen)
model_dir = os.path.join("..", "..", "Models", "Baby Detection and Seat Classification Random Forest")

# Ensure directory exists
os.makedirs(model_dir, exist_ok=True)

# Define model path
model_path = os.path.join(model_dir, "random_forest_model_Baby_Covered_in_Blanket_Sunscreen.pkl")

# --- Check and replace model ---
if os.path.exists(model_path):
    os.remove(model_path)
    print(f"Old model removed: {model_path}")

joblib.dump(clf, model_path)
print(f"New model saved at: {os.path.abspath(model_path)}")

